In [33]:
#import library
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import pandas as pd
from operator import itemgetter
from math import log

In [34]:
#set stopwords language
stop_words = stopwords.words('english')
stemmer = PorterStemmer()

In [50]:
def search(word):
    i = 1
    status = True
    while status:
        data = pd.read_csv('data_film.csv',sep=',')
        data_after = pd.read_csv('post_processing_data_film_cleaned.csv',sep="|")
        
        data_word = word.split(" ")
        
        # query
        query_word = []
        for dw in data_word:
            token = word_tokenize(dw)
            dw_cleaned_token = ""
            for word_token in token:
                if word_token not in stop_words:
                    dw_cleaned_token += word_token +" "
            
            token_stemming = word_tokenize(dw_cleaned_token)
            for ws in token_stemming:
                query_word.append(stemmer.stem(ws))
        
        print('Query Word = ',query_word)
            
        
        # TF - DF
        termFreq = []
        docsFreq = []
        for i, term in enumerate(query_word):
            docsCounter = []
            docsFreqCounter = 0
            for doc in data_after['word']:
                counter = 0
                for word in doc.split(' '):
                    if word == term:
                        counter +=1
                docsCounter.append(counter)
                
                if counter > 0:
                    docsFreqCounter +=1
            termFreq.append(docsCounter)
            docsFreq.append(docsFreqCounter)
        
        # IDF + 1
        idf_one = []
        docsTotal = len(data)
        for i in docsFreq:
            idf_one.append(log(docsTotal)/(i+1))
            
        # TFIDF
        docs_weight = termFreq[:]
        for i in range(len(docs_weight)):
            for j in range(len(docs_weight[i])):
                docs_weight[i][j] = termFreq[i][j]*idf_one[i]
                
        # sum total weight docs
        total_weight = []
        for i in range(len(docs_weight[0])):
            total_weight.append([i,0])
        
        for i in range(len(docs_weight)):
            for j in range(len(docs_weight[i])):
                total_weight[j][1] += docs_weight[i][j]
                
        # sort document relevan
        total_weight = sorted(total_weight,key=itemgetter(1),reverse=True)
        
        # show top 25 document relevan
        for i in range(0,15):
            print(str(i+1)+str('. ')+str(data['title'][total_weight[i][0]])+" | "+str(total_weight[i][1]))
    
        status = False


In [53]:
search("Soldier")

Query Word =  ['soldier']
1. The Falcon and the Winter Soldier | 2.0858620314616054
2. Queen of the South | 2.0858620314616054
3. Hitman's Wife's Bodyguard | 2.0858620314616054
4. Real World/Road Rules Challenge | 2.0858620314616054
5. Godzilla vs. Kong | 0.0
6. Black Widow | 0.0
7. Jupiter's Legacy | 0.0
8. Zack Snyder's Justice League | 0.0
9. Mortal Kombat | 0.0
10. Invincible | 0.0
11. The Suicide Squad | 0.0
12. Game of Thrones | 0.0
13. Thunder Force | 0.0
14. Loki | 0.0
15. Shingeki no kyojin | 0.0
